<a href="https://colab.research.google.com/github/AtSourav/CatsnDogs_VAE/blob/main/CatsnDogs_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We're going to build a VAE in keras with the encoder and decoder based on CNNs.

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import losses
from keras import layers
from keras import utils

In [4]:
input_size = (160,160,3)
latent_dim = 16

In [5]:
encoder_input = keras.Input(shape=input_size)

x = layers.Conv2D(16, 3, padding="valid")(encoder_input)
x = layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(x)    # with strides=None this defaults to pool_size
x = layers.BatchNormalization(axis=-1)(x) # the default data_format in the conv2d is "channels last", we want to normalize across the channels, hence we set axis=-1
x = layers.LeakyReLU()(x)

x = layers.Conv2D(32, 3, padding="valid")(x)
x = layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(64, 3, padding="valid")(x)
x = layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 3, padding="valid")(x)
x = layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)       # dimension of vector produced 8*8*256

# we won't add any extra dense layers for now

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)

def sampling(arg):
  z_m, z_log_v = arg
  batch = tf.shape(z_m)[0]
  dim = tf.shape(z_m)[1]
  eps = tf.random.normal(shape=(batch,dim))
  return z_m + tf.exp(0.5*z_log_v)*eps

z = layers.Lambda(sampling)([z_mean,z_log_var])   # we feed the sampling function in to a Lambda layer to build form a layer for the architecture as keras needs

encoder = keras.Model(encoder_input, [z_mean, z_log_var, z], name='encoder')           # the second argument specifies that the encoder outputs [z_mean, z_log_var, z] for each input.
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 158, 158, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 79, 79, 16)   0           ['conv2d[0][0]']                 
                                                                                            

The number of trainable parameters might just be too big for the dataset, possibility of overtraining, we'll see how it goes. Let's make the decoder now.

In [6]:
latent_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(10*10*128)(latent_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((10,10,128))(x)

x = layers.Conv2DTranspose(64, 3, strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
                                                        # axis=-1 is set by default
x = layers.Conv2DTranspose(32, 3, strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(16, 3, strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

decoder_output = layers.Conv2DTranspose(3, 3, activation='sigmoid', strides=2, padding='same')(x)

decoder = keras.Model(latent_input, decoder_output, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense (Dense)               (None, 12800)             217600    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 12800)             0         
                                                                 
 reshape (Reshape)           (None, 10, 10, 128)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 20, 20, 64)       73792     
 nspose)                                                         
                                                                 
 batch_normalization_4 (Batc  (None, 20, 20, 64)       256       
 hNormalization)                                           

Now let's instantiate the VAE by combininb the encoder and the decoder layers.

In [7]:
decoder_out = decoder(encoder(encoder_input)[2])          # we feed in z from the encoder output
VAE = keras.Model(encoder_input, decoder_out, name='VAE')

VAE.summary()

Model: "VAE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 encoder (Functional)        [(None, 16),              360576    
                              (None, 16),                        
                              (None, 16)]                        
                                                                 
 decoder (Functional)        (None, 160, 160, 3)       315363    
                                                                 
Total params: 675,939
Trainable params: 675,235
Non-trainable params: 704
_________________________________________________________________


We shall define the loss function now, it's made of the reconstruction loss that tries to ensure repoducibility of the data, and the KL divergence loss that tries to ensure that the learned (approximate) posterior is close to the true posterior. Since we have continuous values for the pixels, we cannot use the Bernoulli log loss (binary cross entropy) for the reconstruction loss, we shall use mean squared error. In a different version, we shall try to implement a continuous version of the Bernoulli log loss based on 1907.06845.

We shall keep a relative weight beta between the two terms in the total loss as a hyperparameter. We may want to investigate the effects of varying this hyperparameter eventually.

The custom loss function needs to be a callable of the form fn(true, pred) to be compatible with Model.compile() and Model.fit(). The reconstruction loss will calculate the mse between the true and predicted images directly. The kl divergence loss is a function of z_mean and z_log_var that will be used directly from the values while evaluating the forward pass. For further details, look up the source code for the compile() and fit() methods in the Model class, and also for train_step(), compute_loss(), and make_train_function()

In [8]:
beta = 1  # we shall vary this later

def vae_loss(true,pred):
  mse = losses.MeanSquaredError()
  reconstruction_loss = (160*160*3)*mse(layers.flatten(true),layers.flatten(pred)).numpy()    # the input and the output for the model, which are 160*160*3 images will be taken as true and pred, respectively.

  kl_div = tf.reduce_sum(-0.5*(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)), 1)     # summing along the columns, if we chose to take mean instead of sum at this step, we shouldn't multiply with the dimensions for the reconstruction loss.

  return reconstruction_loss + beta*kl_div

In [9]:
VAE.compile(optimizer='adam',loss=vae_loss)

Now we want to load our dataset, and train the VAE on it. We shall use the cats and dogs img dataset from this [repo](https://github.com/AtSourav/CatsnDogs_VAE.git)

In [10]:
! git clone https://github.com/AtSourav/CatsnDogs_img_dataset.git           # ! let's us use shell commands

Cloning into 'CatsnDogs_img_dataset'...
remote: Enumerating objects: 5745, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 5745 (delta 1), reused 0 (delta 0), pack-reused 5739
Receiving objects: 100% (5745/5745), 44.62 MiB | 31.45 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [11]:
!ls

CatsnDogs_img_dataset  sample_data


We have cloned the repo with the dataset here so we can access it directly.

In [18]:
!ls CatsnDogs_img_dataset/Dataset_aug | sed -n 20p     # -n sed asks not to print every line, 20p specifies that we want it to print the 20th entry

Cat_1018.jpg


We shall now make a train and validation dataset from the images in Dataset_aug folder, and also normalize images to have pixel values between 0 and 1 as opposed to 0 and 255.

In [25]:
batch_size = 128
img_dir = 'CatsnDogs_img_dataset/Dataset_aug/'


data_train = utils.image_dataset_from_directory(img_dir, labels=None, validation_split=0.1, subset="training", seed=111, image_size=(160,160), color_mode='rgb', shuffle=True, batch_size=batch_size)

data_valid = utils.image_dataset_from_directory(img_dir, labels=None, validation_split=0.1, subset="validation", seed=111, image_size=(160,160), color_mode='rgb', shuffle=True, batch_size=batch_size)

for image_batch in data_train:
  print(image_batch.shape)
  break

for image_batch in data_valid:
  print(image_batch.shape)
  break


Found 5110 files belonging to 1 classes.
Using 4599 files for training.
Found 5110 files belonging to 1 classes.
Using 511 files for validation.
(128, 160, 160, 3)
(128, 160, 160, 3)
